# 1. Загрузка и подготовка данных

Отзывы для обучения уже загружены скриптом в SQLite3 бд.

Часть данных была вручную размечена для улучшения качества. Эти датасеты не включены в репозиторий, однако я готов ими поделиться.

In [4]:
import pandas as pd
import numpy as np
import sqlite3
import html
%matplotlib inline

In [5]:
train_data = pd.read_csv("./train_data_other.csv", usecols=["text", "label", "assessed"])
# train_data["label"] = train_data["label"].map({"pos":1,"neg":0})
train_data.head()

,text,label,assessed
0,"Очень хороший телефон, первый из линейки Redmi...",1,0
1,"Одним словом бонба, айфон курит в сторонке. Хо...",1,0
2,Доволен.,1,0
3,превосходно.,1,1
4,Телефон покупал как замену Редми 5+ всем устра...,1,1


In [4]:
train_data.text[0]

'Плюсы: Хорошая камера, получаются четкие снимки в режиме 64мп Производительность Красивый Хороший экран Держит заряд целый день и еще остаётся процентов 30, если не играть в игры Соотношение цена/качество Присутствует модуль NFS . Минусы: Сканер отпечатка пальцев находиться рядом с камерами Вырез капелька Маркий и очень скользкий Убрали возможность записывать звонки, но это не только у этой версии телефона. Впечатления: Очень хороший телефон, первый из линейки Redmi у которого есть NFC, многие бояться процессора MediaTek, но телефон очень производительный, тянет игры на максималках и не сильно греется. Камеры бомба, присутствует даже макрообъектив. Без чехла лучше не носить, скользкий и легко уронить. В общем достойная модель, учитывая что на алике можно заказать за 14к .'

## 1.1 Подготовка текста

Попробую аугментировать мой набор данных. Специфика такая, что в отзыв могут написать как хорошее, так и плохое (для этого есть отдельные секции). Две идеи

1. Добавить отзывы, состоящие только из содержимого секций "Плюсы" и "Минусы" с соответствующими лейблами
2. Добавить отзывы, в которых, в зависимости от известного лейбла, убрано содержимое противоположной секции

In [9]:
separated_chunks = []
for _, row in train_data.iterrows():
    try:
        _positive, _remaining = row.text.split("Минусы: ")
        _negative, _remaining = _remaining.split("Впечатления: ")
        _positive = _positive.replace("Плюсы: ", "")
        separated_chunks.append({"positive": _positive, "negative": _negative, "other": _remaining, "label": row.label})
    except ValueError:
        continue


In [10]:
len(separated_chunks)

10151

In [12]:
separated_chunks[50]

{'positive': '1) Экран. 2) Корпус из металла. ',
 'negative': '1) Самый жирный - система IOS (как в тюрьме или в армии) 2) Цена высокая 3) Скользкий 4) За такую цену качество должно превосходить все смартфоны, но увы - наоборот. . ',
 'other': 'Впечатлений масса, но все они не в пользу этого смартфона (ипхона).',
 'label': 1}

In [13]:
aug_data_synthetic_labels = []
for chunk in separated_chunks:
    aug_data_synthetic_labels.append({"text": chunk["positive"], "label": 1})
    aug_data_synthetic_labels.append({"text": chunk["negative"], "label": 0})

In [14]:
aug_data_irrelevant_separated = []
for chunk in separated_chunks:
    aug_data_synthetic_labels.append({
        "text": chunk["positive"] + " " + chunk["other"] if chunk["label"] == 1 else chunk["negative"] + " " + chunk["other"], 
        "label": chunk["label"]
        })


In [40]:
aug_data_feedback_only = []
for chunk in separated_chunks:
    aug_data_feedback_only.append({
        "text": chunk["other"], 
        "label": chunk["label"]
        })
train_data = pd.DataFrame(aug_data_feedback_only)

In [15]:
train_data = train_data.append(pd.DataFrame(aug_data_synthetic_labels))

In [16]:
train_data = train_data.append(pd.DataFrame(aug_data_irrelevant_separated))

In [4048]:

for (idx, row) in train_data[train_data.assessed == 0].iterrows():
    print(f"проверено: {sum(train_data.assessed)}    осталось {len(train_data) - sum(train_data.assessed)}")
    print(f"idx: {idx}")
    print(row.label)
    print(row.text)
    train_data.at[idx, 'assessed'] = 1
    break


проверено: 7751    осталось 2400
idx: 6029
1
Отличная модель за приемлемую цену..


In [4002]:
train_data.at[idx, 'label'] = 0

In [4020]:
train_data.at[idx, 'label'] = 1

In [5056]:
idx = randomset[-2]
print(train_data.iloc[_rn].label)
print(train_data.iloc[_rn].text)

0
В эксплуатации почти 1 год. Аппарат не плохой. Работает стабильно. Батарея держит хорошо. До этого сидел на яблоке. Если сравнивать, то система iOS это премиум, андроид это эконом. От яблока отказался - за долбали обновления и умышленные действия яблока направленные на покупку новых аппаратов . .


In [4049]:
train_data.to_csv("train_data_other.csv")

In [2515]:
train_data.iloc[1261]

text        В целом модель то не плохая, но с каждым обнов...
label                                                       0
assessed                                                    1
Name: 1261, dtype: object

Что касается предобработки, то я буду использовать два варианта: довольно стерильные наборы слов и +- оригинальные данные.

In [2516]:
from sklearn.base import TransformerMixin, BaseEstimator

In [3227]:
import re
import html

class RoughPreprocessor(TransformerMixin):
    def __init__(self):
        pass
    
    def fit_transform(self, data, y=None):
        return list(map(self.normalize_text_re, map(self.normalize_text, data)))
    
    def normalize_text(self, text):
        _t = html.unescape(text)
        _t = _t.replace("Плюсы: ",". ")
        _t = _t.replace("Минусы: ",". ")
        _t = _t.replace("Впечатления: ",". ")
        _t = _t.replace("<p>"," ")
        _t = _t.replace("</p>", " ")
        _t = _t.replace("\n", " ")
        _t = _t.replace("\r", " ")
        _t = _t.replace("\t", " ")
        _t = _t.replace('"', " ")
        _t = _t.lower()
        return _t.strip()
    
    def normalize_text_re(self, text):
        _t = text
        _t = re.sub(r"[\s.,\-\+><;:!?()]", " ", _t)
        _t = re.sub(r"\s+", " ", _t)
        return _t.strip()
    
    def fit(self, data, y=None):
        return self.fit_transform(data)
    
    def transform(self, data, y=None):
        return self.fit_transform(data) 

In [19]:
# dummy препроцессор, чтоб проще пайплайн строить

In [3228]:
class DummyPreprocessor(TransformerMixin):
    def __init__(self):
        pass
    
    def fit_transform(self, data, y=None):
        return data
    
    def transform(self, data, y=None):
        return data
    
    def fit(self, data, y=None):
        return data

## 2. Векторизация

Я хочу воспользоваться библиотекой gensim, потому что я уже использовал их эмбеддинги, и они отлично показали себя в классификации, даже на плохо обработанном датасете.

Чтобы потом удобно запаковать это в sklearn pipeline, я реализую свой класс по образу `TfIdfVectorizer` из `sklearn.feature_extraction`

Также я попробую TfIdfVectorizer и CountVectorizer сам по себе для сравнения какой лучше

In [2519]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [474]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models import doc2vec
from tqdm import tqdm

import numpy as np
import re
import html

class GensimVectorizer(BaseEstimator):
    def __init__(self):
        pass

    def fit(self, raw_documents, y=None):
        X = self.preprocess(raw_documents)
        # print("Creating model...")
        model = gensim.models.doc2vec.Doc2Vec(
            vector_size=100, 
            min_count=10,
            epochs=40
        )
        # print("Building vocab...")
        model.build_vocab(X)
        # print("Training doc2vec...")
        model.train(X, total_examples=model.corpus_count, epochs=model.epochs)
        self.model = model
        return self

    def transform(self, raw_documents, y=None):
        X = self.preprocess(raw_documents)
        # print("Iinferring vectors...")
        vectorized_texts = []
        for doc_id, _ in enumerate(tqdm(X, desc="Inferring vectors: ")):
            inferred_vector = self.model.infer_vector(X[doc_id].words)
            vectorized_texts.append(inferred_vector)

        return vectorized_texts

    def preprocess(self, raw_documents):
        # print("Tokenization...")
        processed_texts = []
        for idx, text in enumerate(tqdm(raw_documents, desc="Tokenization: ")):
            processed_texts.append(doc2vec.TaggedDocument(simple_preprocess(text), [idx]))
        return processed_texts


    def fit_transform(self, texts, y=None) -> np.ndarray:
        self.fit(texts)
        X = self.transform(texts)
        
        return np.array(X)

## 3. Обучение модели 

Модели, которые я буду рассматривать:
+ LogisticRegression
+ GradientBoosting
+ LinearSVC
+ BayesianClassifier

In [3229]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [4050]:
X = train_data["text"]
# y = data.label.map({"pos": 1, "neg": 0})
y = train_data.label

In [2522]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate

_model = make_pipeline(DummyPreprocessor(), CountVectorizer(ngram_range=(1,3)), LinearSVC(max_iter=4000, class_weight="balanced"))

In [4051]:
_model.fit(X, y)

Pipeline(steps=[('dummypreprocessor',
                 <__main__.DummyPreprocessor object at 0x7ff63a088ac0>),
                ('countvectorizer', CountVectorizer(ngram_range=(1, 3))),
                ('linearsvc',
                 LinearSVC(class_weight='balanced', max_iter=4000))])

In [4052]:
cross_val_score(_model, X, y, n_jobs=6, cv=4)

array([0.9034673 , 0.90464933, 0.90031521, 0.86677178])

## 4. Улучшение модели

Улучшить модель можно несколькими путями:
+ 1. Попробовать разный препроцессинг текста
+ 2. Подобрать параметры модели векторизации
+ 3. Подобрать параметры классификатора
+ 4. Попробовать другие классификаторы
+ 5. Сбалансировать классы для обучения
+ 0. Делать перебор не по сетке, а более "разумными" методами

Я сделаю только пп. 1, 3

In [3231]:
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingRandomSearchCV, ParameterGrid

In [3232]:

from sklearn.pipeline import Pipeline
from scipy.stats import randint

pipeline_ = Pipeline([
    ("preprocessor", RoughPreprocessor()),
    ("vec", GensimVectorizer()),
    ("clf", GradientBoostingClassifier())
])
# я разбил на несколько словарей потому что 
# параметры не унифицированы между классами моделей
linear_classifiers = {
    "clf":[LogisticRegression(), LinearSVC()],
    "clf__class_weight":["balanced"],
    # "clf__dual": [True, False]
}
bayesian_classifiers = {
    "clf": [GaussianNB()],
}
forest_classifiers = {
    "clf":[GradientBoostingClassifier(), RandomForestClassifier()],
    "clf__n_estimators":randint(100, 500),
    "clf__max_depth": randint(3, 20),
}
preprocessors = {
    "preprocessor":[RoughPreprocessor(), DummyPreprocessor()],
}
common_vectorizers = {
    "vec": [CountVectorizer(), TfidfVectorizer()],
    "vec__ngram_range": [(1,1), (1,2), (1,3), (1,4), (1,5)],
    "vec__min_df": randint(1, 10),
    "vec__max_features": [None, 200, 500, 1000, 350, 1500]
}
embedding_vectorizers = {
    "vec": [GensimVectorizer()]
}

param_distributions = [
{
    # Plain Vectorizers + Linear Models
    **preprocessors,
    **common_vectorizers,
    **linear_classifiers    
}, 
# {
#     # Plain Vectorizers + Linear Models
#     **preprocessors,
#     **common_vectorizers,
#     **bayesian_classifiers    
# },
# {
#     # Gensim Embeddings + Linear Models
#     **preprocessors,
#     **embedding_vectorizers,
#     **linear_classifiers,
# }, 
# {
#     # Gensim Embeddings + GradBoost Models
#     **preprocessors,
#     **embedding_vectorizers,
#     **forest_classifiers,
# }, 
# {
#     # Plain Vectorizers + GradBoost Models
#     **preprocessors,
#     **common_vectorizers,
#     **forest_classifiers,
# }
]
    

In [4053]:
%%time
# ~ несколько недель на 6 ядрах Ryzen5
hrscv_ = HalvingRandomSearchCV(
    estimator=pipeline_,
    param_distributions=param_distributions, 
    scoring="f1", 
    n_jobs=6, refit=True, cv=3, verbose=2)
hrscv_.fit(X, y)

n_iterations: 7
n_required_iterations: 7
n_possible_iterations: 7
min_resources_: 12
max_resources_: 10151
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 845
n_resources: 12
Fitting 3 folds for each of 845 candidates, totalling 2535 fits
----------
iter: 1
n_candidates: 282
n_resources: 36
Fitting 3 folds for each of 282 candidates, totalling 846 fits
----------
iter: 2
n_candidates: 94
n_resources: 108
Fitting 3 folds for each of 94 candidates, totalling 282 fits
----------
iter: 3
n_candidates: 32
n_resources: 324
Fitting 3 folds for each of 32 candidates, totalling 96 fits
----------
iter: 4
n_candidates: 11
n_resources: 972
Fitting 3 folds for each of 11 candidates, totalling 33 fits
----------
iter: 5
n_candidates: 4
n_resources: 2916
Fitting 3 folds for each of 4 candidates, totalling 12 fits
----------
iter: 6
n_candidates: 2
n_resources: 8748
Fitting 3 folds for each of 2 candidates, totalling 6 fits
CPU times: user 5.08 s, sys: 269 ms, total: 5.34 s
W

HalvingRandomSearchCV(cv=3,
                      estimator=Pipeline(steps=[('preprocessor',
                                                 <__main__.RoughPreprocessor object at 0x7ff639e73280>),
                                                ('vec', GensimVectorizer()),
                                                ('clf',
                                                 GradientBoostingClassifier())]),
                      n_jobs=6,
                      param_distributions=[{'clf': [LogisticRegression(class_weight='balanced'),
                                                    LinearSVC(class_weight='balanced')],
                                            'clf__class_weight': ['balanced'],
                                            'preprocesso...
                                            'vec': [CountVectorizer(min_df=7,
                                                                    ngram_range=(1,
                                                                    

Лучший набор параметров:


In [4054]:
print(hrscv_.best_score_)
print(hrscv_.best_params_)

0.8852723710256737
{'clf': LinearSVC(class_weight='balanced'), 'clf__class_weight': 'balanced', 'preprocessor': <__main__.RoughPreprocessor object at 0x7ff639e73d90>, 'vec': TfidfVectorizer(max_features=1500, min_df=9, ngram_range=(1, 3)), 'vec__max_features': 1500, 'vec__min_df': 9, 'vec__ngram_range': (1, 3)}


In [ ]:
0.9819234084472356
{'clf': GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), 'clf__max_depth': 10, 'clf__n_estimators': 500, 'preprocessor': <__main__.DummyPreprocessor object at 0x7fe9618d6d30>, 'vec': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=1000, min_df=2,
                ngram_range=(1, 4), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None), 'vec__max_features': 1000, 'vec__min_df': 2, 'vec__ngram_range': (1, 4)}

In [4066]:
pipelne_final = hrscv_.best_estimator_
classifier_final = hrscv_.best_estimator_["clf"]
vectorizer_final = hrscv_.best_estimator_["vec"]

<1x1500 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [3213]:
model_final = _model

## 5. Инференс и подготовка сабмита

In [486]:
from sklearn.metrics import accuracy_score

In [41]:
# это для импорта предоставленного файла
import bs4
test = []
with open("test.csv") as tfile:
    sp = bs4.BeautifulSoup(tfile)
    revs = sp.findAll("review")
    for r in revs:
        test.append(r.text)

# pd.read_csv("test.csv")

In [487]:
# а это мой размеченный тестовый файл
test = pd.read_csv("./test_.csv", usecols=["text", "y"])

In [488]:
test.head()

,text,y
0,"Ужасно слабый аккумулятор, это основной минус ...",0
1,ценанадежность-неубиваемостьдолго держит батар...,1
2,"подробнее в комментариях\nК сожалению, факт по...",0
3,я любительница громкой музыки. Тише телефона у...,0
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех...",1


In [4056]:
test["prediction"] = model_final.predict(test.text)

In [4057]:
accuracy_score(test["y"], test["prediction"])


0.76

In [54]:
# моя разметка тестового файла не очень точная.
accuracy_score(test["y"], test["prediction"])

0.8

In [55]:
submission = test.copy()
submission["y"] = test.prediction.map({1: "pos", 0: "neg"})
submission.to_csv("./submission.csv", columns=["y"], index_label="Id")

## 6. Упаковка модели

In [3242]:
import pickle as pkl
import dill
from sklearn.externals import joblib

In [4068]:
with open("../models/SentimentModelRU.pkl", "wb") as fout:
    dill.dump(classifier_final, fout)
with open("../models/VectorizerRU.pkl", "wb") as fout:
    dill.dump(vectorizer_final, fout)

In [4069]:
with open("../models/SentimentModelRU.pkl", "rb") as fin:
    classifier_ = dill.load(fin)
with open("../models/VectorizerRU.pkl", "rb") as fin:
    vectorizer_ = dill.dump(fin)

TypeError: file must have 'read' and 'readline' attributes

In [3243]:
joblib.dump(final_pipeline, "../SentimentModelRU.pkl")

['../SentimentModelRU.pkl']